Using `pykeen.ablation` instead of `pykeen.hpo` (https://pykeen.readthedocs.io/en/stable/tutorial/running_ablation.html)

In [1]:
from pykeen.ablation import ablation_pipeline
import json

output_dir = "./test_ablation_results"

In [65]:
models = ["RotatE", "TransE"]

ablation_config = {
    "metadata": {
        "title": "Ablation Study Over Nations for ComplEx."
    },
    "ablation": {
        #"datasets": ["nations"],
        "epochs": 100,
        "models":  models,
        "losses": ["MarginRankingLoss", "NSSALoss"], # sec 7.4 'impact of the loss function'
        "training_loops": ["sLCWA"], # just use sLCWA as LCWA expensive (paper sec. 7 -> training approaches)
        "optimizers": ["adam"],
        "negative_sampler": "BasicNegativeSampler",
        "create_inverse_triples": [True, False],
        "stopper": "early",
        "stopper_kwargs": {
            "frequency": 10,
            "patience": 20,
            "relative_delta": 0.002,
            "metric": "hits@10"
        },
        "model_to_model_kwargs_ranges": {
            model: {
                "embedding_dim": {
                    "type": "int", 
                    "low": 6, # = 64
                    "high": 8, # = 256
                    "scale": "power_two"
                }
            } for model in models
        },
        "model_to_training_loop_to_training_kwargs_ranges": {
            model: {
                "sLCWA": {
                    "batch_size": {
                        "type": "int",
                        "low": 10, # 1024
                        "high": 13, # 8192
                        "scale": "power_two"
                    }
                }
            } for model in models
        },
        "model_to_optimizer_to_optimizer_kwargs_ranges": {
            model : {
                "adam": {
                    "lr": {
                        "type": "float",
                        "low": 0.001,
                        "high": 0.1,
                        "scale": "log"
                    }
                }
            } for model in models
        },
        "model_to_loss_to_loss_kwargs_ranges": {
            model: {
                "MarginRankingLoss": {
                    "margin": {
                        "type": "categorical",
                        "choices": [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5],
                    }
                },
                "NSSALoss": {
                    "margin": {
                        "type": "int",
                        "low": 1,
                        "high": 29,
                        "q": 2
                    },
                    "adversarial_temperature": {
                        "type": "float",
                        "low": 0.1,
                        "high": 1,
                        "q": 0.1
                    }
                }
            } for model in models
        },
        "model_to_negative_sampler_to_negative_sampler_kwargs_ranges": {
            model: {
                "basic": {
                    "num_negs_per_pos": {
                        "type": "int",
                        "low": 1,
                        "high": 50,
                        "q": 1
                    }
                }
            } for model in models
        }
    },
    "optuna": {
        "n_trials": 2,
#         "timeout": 300,
        "metric": "hits@10",
        "direction": "maximize",
        "sampler": "random",
        "pruner": "nop"
        }
}

In [66]:
ablation_pipeline("nations", "./ablation_tests_2" , **ablation_config["ablation"], **ablation_config["optuna"])

INFO:pykeen.ablation.ablation:Dataset: nations
INFO:pykeen.ablation.ablation:Add inverse triples: True
INFO:pykeen.ablation.ablation:Model: RotatE
INFO:pykeen.ablation.ablation:Loss functions: MarginRankingLoss
INFO:pykeen.ablation.ablation:Optimizer: adam
INFO:pykeen.ablation.ablation:Training loop: sLCWA
INFO:pykeen.ablation.ablation:Negative sampler: BasicNegativeSampler
INFO:pykeen.ablation.ablation:Evaluator: None
INFO:pykeen.ablation.ablation:Dataset: nations
INFO:pykeen.ablation.ablation:Add inverse triples: True
INFO:pykeen.ablation.ablation:Model: RotatE
INFO:pykeen.ablation.ablation:Loss functions: NSSALoss
INFO:pykeen.ablation.ablation:Optimizer: adam
INFO:pykeen.ablation.ablation:Training loop: sLCWA
INFO:pykeen.ablation.ablation:Negative sampler: BasicNegativeSampler
INFO:pykeen.ablation.ablation:Evaluator: None
INFO:pykeen.ablation.ablation:Dataset: nations
INFO:pykeen.ablation.ablation:Add inverse triples: True
INFO:pykeen.ablation.ablation:Model: TransE
INFO:pykeen.abla

int power_two embedding_dim 64
categorical None margin 5.5
categorical None margin_activation softplus
float log lr 0.06335807923102385
<class 'int'> None num_negs_per_pos 10
num_epochs not suggested
int power_two batch_size 2048


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/2 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/2 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
INFO:pykeen.evaluation.evaluator:Evaluation took 0.01s seconds


Training batches on cpu:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/2 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [56]:
with open("ablation_config.json", "w") as f:
    json.dump(ablation_config, f)

!pykeen experiments ablation ./ablation_config.json -d ./ablation_tests 

#--dry-run


2021-07-06 16:14:43 INFO     Dataset: nations
2021-07-06 16:14:43 INFO     Add inverse triples: True
2021-07-06 16:14:43 INFO     Model: RotatE
2021-07-06 16:14:43 INFO     Loss functions: MarginRankingLoss
2021-07-06 16:14:43 INFO     Optimizer: adam
2021-07-06 16:14:43 INFO     Training loop: sLCWA
2021-07-06 16:14:43 INFO     Negative sampler: BasicNegativeSampler
2021-07-06 16:14:43 INFO     Evaluator: None
2021-07-06 16:14:43 INFO     Dataset: nations
2021-07-06 16:14:43 INFO     Add inverse triples: True
2021-07-06 16:14:43 INFO     Model: RotatE
2021-07-06 16:14:43 INFO     Loss functions: NSSALoss
2021-07-06 16:14:43 INFO     Optimizer: adam
2021-07-06 16:14:43 INFO     Training loop: sLCWA
2021-07-06 16:14:43 INFO     Negative sampler: BasicNegativeSampler
2021-07-06 16:14:43 INFO     Evaluator: None
2021-07-06 16:14:43 INFO     Dataset: nations
2021-07-06 16:14:43 INFO     Add inverse triples: True
2021-07-06 16:14:43 INFO     Model: TransE
2021-07-06 16:14:43 INFO     Loss f